In [9]:
from datetime import timedelta
import cv2
import numpy as np
import os
import time
import glob
import random
import shutil
import pathlib

In [2]:
SAVING_FRAMES_PER_SECOND = 10

In [3]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return "-" + result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"-{result}.{ms:02}".replace(":", "-")

In [4]:
def get_saving_frames_durations(cap, saving_fps):
    """Функция, которая возвращает список длительностей сохраняемых кадров"""
    s = []
    # получаем длительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # используем np.arange() для выполнения шагов с плавающей запятой
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [5]:
def extract_frames(video_file, save_dest=None, id=1):
    if save_dest == None:
        save_dest, _ = os.path.splitext(video_file)
        save_dest += "_frames"
    # создаем папку по названию видео файла
    if not os.path.isdir(save_dest):
        os.mkdir(save_dest)
    # читать видео файл    
    cap = cv2.VideoCapture(video_file)
    # получить FPS видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    # если наше SAVING_FRAMES_PER_SECOND больше FPS видео, то установливаем минимальное
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей кадров для сохранения
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # начало цикла
    count = 0
    save_count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            break
        # получаем длительность, разделив текущее количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую первоначальную длительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуст, все кадры сохранены
            break
        if frame_duration >= closest_duration:
            # если ближайшая длительность меньше или равна длительности текущего кадра,
            # сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            saveframe_name = os.path.join(save_dest, f"video{id}_frame{frame_duration_formatted}.jpg")
            cv2.imwrite(saveframe_name, frame)
            save_count += 1
            print(f"{saveframe_name} сохранён")
            # удалить текущую длительность из списка, так как этот кадр уже сохранен
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить счечик кадров count
        count += 1
        
    print(f"Итого сохранено кадров {save_count}")

In [14]:
BASE_DIR = pathlib.Path('Extract_Frames.ipynb').resolve(strict=True).parent.parent.parent
save_path = BASE_DIR.joinpath('Data', 'Train')
videos_dir = BASE_DIR.joinpath('Videos')

In [15]:
#extract_frames('Videos\Flux_Heat\Flux_Heat.mp4', 'Data\Train1\Flux_Heat')

class_names=os.listdir(videos_dir)
for name in class_names:
    vid_id = 1 
    save_to = os.path.join(save_path, name)
    video_class = os.path.join(videos_dir, name)
    for video in glob.glob(os.path.join(video_class, '*.mp4')):
        print(video)
        begtime = time.perf_counter()
        extract_frames(video, save_to, vid_id)
        endtime = time.perf_counter()
        print(f"\nЗатрачено, с: {endtime - begtime} ")
        vid_id += 1

C:\Users\Pseud\Documents\Python Scripts\Observer\Videos\Flux_Heat\Flux_Heat.mp4
29.97002997002997
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0:00:00.00.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0-00-00.10.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0-00-00.20.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0-00-00.30.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0-00-00.40.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0-00-00.50.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0-00-00.60.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observer\Data\Train\Flux_Heat\video1_frame-0-00-00.70.jpg сохранён
C:\Users\Pseud\Documents\Python Scripts\Observ

In [22]:
test_dir = os.path.join('Data', 'Test1')
dirs = os.listdir(save_path)
for dir in dirs:
    class_dir = os.path.join(save_path, dir)
    cd_list = os.listdir(class_dir)
    amount = len(cd_list)
    test_sample = random.sample(cd_list, int(amount*0.1))
    os.mkdir(os.path.join(test_dir, dir))
    for sample in test_sample:
        print(sample)
        source = os.path.join(class_dir, sample)
        print(source)
        destination = os.path.join(test_dir, dir)
        shutil.move(source, destination)


video1_frame-0-00-09.51.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-09.51.jpg
video1_frame-0-00-19.72.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-19.72.jpg
video1_frame-0-00-09.01.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-09.01.jpg
video1_frame-0-00-32.53.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-32.53.jpg
video1_frame-0-00-20.32.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-20.32.jpg
video1_frame-0-00-32.63.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-32.63.jpg
video1_frame-0-00-19.02.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-19.02.jpg
video1_frame-0-00-17.32.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-17.32.jpg
video1_frame-0-00-08.01.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-08.01.jpg
video1_frame-0-00-10.51.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-10.51.jpg
video1_frame-0-00-22.52.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-22.52.jpg
video1_frame-0-00-01.00.jpg
Data\Train1\Flux_Heat\video1_frame-0-00-01.00.jpg
video1_frame-0-00-17.02.jpg
Data\Train1\Flux_Heat\video1_frame-0